In [ ]:
# Imports
import os
import re
import json
import requests
import subprocess
from typing import Optional
from pathlib import Path
from pydantic import BaseModel

### Prepare a new Git repository

In [ ]:
dirname = Path("output")

In [ ]:
dirname.mkdir(parents=True, exist_ok=True)

In [ ]:
# Make Git ignore the global configuration files
os.environ['GIT_CONFIG_SYSTEM'] = "/dev/null"
os.environ['GIT_CONFIG_GLOBAL'] = "/dev/null"

In [ ]:
subprocess.run(("git", "init", "--initial-branch=main"), cwd=dirname, check=True)

In [ ]:
_ = subprocess.run(("git", "config", "user.name", "AI-Nix-Packager"), cwd=dirname, check=True)
_ = subprocess.run(("git", "config", "user.email", ""), cwd=dirname, check=True)

### Initialize a new Git branch

In [ ]:
# TODO: don't hardcode
#project_name = "findtow" # https://git.sr.ht/~martijnbraam/findtow
project_name = "gResistor" # https://github.com/stethewwolf/gResistor
#project_name = "postmarketos-tweaks" # https://gitlab.postmarketos.org/postmarketOS/postmarketos-tweaks

In [ ]:
_ = subprocess.run(("git", "checkout", "--orphan", project_name), cwd=dirname, check=True)
_ = subprocess.run(("git", "rm", "-rf", "."), cwd=dirname, check=False) # Ignore failures that happen when there are no files

### Generate the default.nix file using nix-init

In [ ]:
# Now please run nix-init inside of the output directory
nix_file = dirname / "default.nix"
if not os.path.isfile(nix_file):
    if not os.path.isfile("default.nix"):
        raise Exception("default.nix not found, please run nix-init inside of the output directory.")
    else:
        raise Exception("Make sure to run nix-init inside of the output directory, not inside the root of the project!")

In [ ]:
def git_commit(message):
    subprocess.run(("git", "add", "--all"), cwd=dirname, check=True)
    subprocess.run(("git", "commit", "-m", message), cwd=dirname, check=True)

In [ ]:
git_commit("Run nix-init")

In [ ]:
# TODO: run nix-format after every step

In [ ]:
# TODO: pin Nixpkgs revision to the one used by this flake
def nix_build():
    return subprocess.run(
        (
            "nix-build",
            "--no-out-link",
            "--log-format", "internal-json",
             "-E", "(import <nixpkgs> { }).callPackage ./. { }"
        ),
        capture_output=True,
        encoding='utf-8',
        cwd=dirname,
        check=False,
        env={
            "PATH": os.getenv("PATH"),
            "NIX_PATH": os.getenv("NIX_PATH"),
            "NO_COLOR": "", # TODO: debug Nix why this has no effect when --log-format is internal-json
            "NIXPKGS_ALLOW_UNFREE": "1",
        },
    )

In [ ]:
build_result = nix_build()

In [ ]:
# Regex from https://stackoverflow.com/a/14693789
ansi_escape = re.compile(r'\x1B(?:[@-Z\\-_]|\[[0-?]*[ -/]*[@-~])')

def parse_error_from_nix_output(output):
    error_messages = []
    for line in output.split("\n"):
        if line == "":
            continue
        prefix = "@nix "
        assert line.startswith(prefix), f"Line does not start with @nix : {line}"
        line = line.lstrip(prefix)
        parsed = json.loads(line)
        assert "action" in parsed, f"Nix JSON output does not have an action key: {parsed}"
        if parsed["action"] != "msg":
            continue
        assert "msg" in parsed, f"Nix JSON output does not have a msg key: {parsed}"
        if parsed["msg"].startswith("fetching path input "):
            continue
        if "raw_msg" in parsed:
            error_message = parsed["raw_msg"]
            derivation_path_search = re.search('For full logs, run:\n  nix log (/nix/store/.*.drv)', error_message)
            if derivation_path_search:
                derivation_path = derivation_path_search.group(1)
                # TODO: teach nix log to respect the NO_COLOR environment variable
                error_message = subprocess.run(("nix", "log", derivation_path), capture_output=True, encoding='utf-8', cwd=dirname, check=True).stdout
            error_message = ansi_escape.sub('', error_message)
            error_messages.append(error_message)
    num_error_messages = len(error_messages)
    assert num_error_messages == 1, f"Unexpected number of error messages. Expected 1, found {num_error_messages}"
    return error_messages[0]

In [ ]:
# Analyze and filter output

if build_result.returncode == 0:
    print("The nix-build command succeeded 🎉")
else:
    print("The nix-build command failed, let's see what the error was.")
    print(build_result.stdout)
    error_message = parse_error_from_nix_output(build_result.stderr)
    print(error_message)

In [ ]:
# TODO: find instances of the absolute path to the current directory and rewrite them
# TODO: special-case errors like evaluation aborted with the following error message:
# 'lib.customisation.callPackageWith: Function called without required argument "libblkid" at /home/u/Dokumente/repos/RAG-with-Langchain/output/default.nix:8'
# to provide some help to the LLM
# TODO: validate syntax and help the model if it is wrong
# TODO: detect if the model ever outputs the same thing twice and tell the model
# TODO: detect if the error message is the same as before and tell the model that the suggestion did not fix the error
# TODO: take existing derivations from Nixpkgs and modify them, e.g. by deleting lines. Then fine-tune an LLM with the error messages
# TODO: prompt the model to tell if thinks that a dependency is missing and then use something like search.nixos.org but lokal (maybe nix-index) to search for it, then tell the model which dependency to add
# TODO: Run multiple different models at each step and compare
# TODO: detect changes that the LLM should not make, such as changing the hash and revert them
# TODO: if the model does not support reasoning, first ask the model for suggestions on fixing the error, then asking it to implement those suggestions

In [ ]:
print("Dependencies loaded successfully!")

In [ ]:
# --- 1. Define Pydantic Models for structured data ---

# Define the model for the LLM's response.
# This helps ensure the LLM's output is consistently parsable.
class NixFixResponse(BaseModel):
    corrected_nix_code: str
    explanation: Optional[str] = "No explanation provided."

print("Pydantic models defined.")

In [ ]:
# --- 2. Configuration and Helper Functions ---

OLLAMA_API_URL = "http://localhost:11434/api/generate"
MODEL_NAME = "gabegoodhart/granite-code:3b-128k"  # Use the model you have running on Ollama

def read_file_content(file_path: str) -> str:
    """Reads the content of a file."""
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")
    with open(file_path, 'r') as f:
        return f.read()

def write_file_content(file_path: str, content: str):
    """Writes content to a file."""
    with open(file_path, 'w') as f:
        f.write(content)
    print(f"File '{file_path}' has been updated.")

def nix_build_and_get_error(file_path: str) -> tuple[bool, str]:
    """
    Runs 'nix-build' on the specified file and captures the error message.
    """
    print(f"Running 'nix-build' on {file_path}...")
    
    dirname = os.path.dirname(file_path)
    if not dirname:
      dirname = "."
    
    # This is the real subprocess call.
    build_result = subprocess.run(("nix-build", "--no-out-link", "--log-format", "internal-json", "-E", "(import <nixpkgs> { }).callPackage ./. { }"), capture_output=True, encoding='utf-8', cwd=dirname, check=False)
    
    error_message = ""
    ansi_escape = re.compile(r'\x1B(?:[@-Z\\-_]|\[[0-?]*[ -/]*[@-~])')
    
    if build_result.returncode == 0:
        print("The nix-build command succeeded 🎉")
        return True, ""
    else:
        print("The nix-build command failed, let's see what the error was.")
        # print(build_result.stdout) # Uncomment for debugging
        for line in build_result.stderr.split("\n"):
            if line == "":
                continue
            prefix = "@nix "
            if not line.startswith(prefix):
                print(f"Warning: line does not start with expected prefix: {line}")
                continue
            line = line.lstrip(prefix)
            
            try:
                parsed = json.loads(line)
            except json.JSONDecodeError:
                print(f"Failed to parse JSON line: {line}")
                continue
            
            if "action" not in parsed:
                print(f"Nix JSON output does not have an action key: {parsed}")
                continue

            if parsed["action"] != "msg":
                continue
            
            if "msg" not in parsed:
                print(f"Nix JSON output does not have a msg key: {parsed}")
                continue

            if parsed["msg"].startswith("fetching path input "):
                continue
            
            if "raw_msg" in parsed:
                error_message = parsed["raw_msg"]
                derivation_path_search = re.search(r'For full logs, run:\n\s+nix log (/nix/store/.*\.drv)', error_message)
                if derivation_path_search:
                    derivation_path = derivation_path_search.group(1)
                    try:
                        # Call 'nix log' to get the full log and replace the short error
                        error_message = subprocess.run(("nix", "log", derivation_path), capture_output=True, encoding='utf-8', cwd=dirname, check=True).stdout
                    except subprocess.CalledProcessError as e:
                        print(f"Failed to get full log from 'nix log': {e}")
                        
                error_message = ansi_escape.sub('', error_message)
                
                # Write the error message to the log file as requested
                error_log_path = os.path.join(dirname, "error.log")
                write_file_content(error_log_path, error_message)
                
                print("Error message parsed and saved to error.log.")
                return False, error_message
    
    return False, "An error occurred, but no message could be parsed."

print("Helper functions and configuration are set.")

In [ ]:
# --- 3. LLM Interaction Logic ---

def get_llm_fix(default_nix_content: str, error_message: str) -> Optional[NixFixResponse]:
    """
    Sends the Nix file and error message to the Ollama model for a fix.
    """
    prompt = f"""
    You are an expert NixOS developer and debugger.
    The following is a default.nix file that failed to compile.
    Your task is to fix the Nix file based on the provided error message.

    Nix file content:
    ```nix
    {default_nix_content}
    ```

    Error message from `nix-build`:
    ```
    {error_message}
    ```

    Please provide the corrected, full Nix file content. Do not include any other text or explanations, just the raw code block. If you can, provide a brief explanation of the fix in a separate JSON field.

    Please respond with a JSON object in this format:
    {{
        "corrected_nix_code": "...",
        "explanation": "..."
    }}
    """

    print("Sending prompt to Ollama...")
    payload = {
        "model": MODEL_NAME,
        "prompt": prompt,
        "format": "json",
        "stream": False,
    }
    
    try:
        response = requests.post(OLLAMA_API_URL, json=payload)
        response.raise_for_status()
        
        raw_response = response.json()
        print("Received response from Ollama.")
        
        # Ollama sometimes returns a string, so we need to parse it.
        # It's important to make the LLM follow the JSON format strictly.
        llm_response_content = json.loads(raw_response['response'])
        
        # Validate the response using Pydantic
        return NixFixResponse(**llm_response_content)
        
    except requests.exceptions.RequestException as e:
        print(f"Error communicating with Ollama: {e}")
        return None
    except (json.JSONDecodeError, KeyError) as e:
        print(f"Failed to parse LLM response: {e}")
        print(f"Raw response: {raw_response}")
        return None

print("LLM interaction logic is ready.")

In [ ]:
# --- 4. Main Program Loop ---

# Create a directory for the output files
output_dir = "output"
os.makedirs(output_dir, exist_ok=True)

# # Create dummy files for demonstration within the new directory
initial_nix_path = "output/default.nix"
error_log_path = "output/error.log"
    
try:
    # Step 1: Read the problematic file and its error
    default_nix_content = read_file_content(initial_nix_path)
    error_message = read_file_content(error_log_path)
    
    print("\n--- Initial State ---")
    print("Original default.nix:\n", default_nix_content)
    print("Compilation Error:\n", error_message)
    
    # Step 2: Simulate a build failure and get the error message
    success, error_message = nix_build_and_get_error(initial_nix_path)
    
    if not success and error_message:
        
        # Step 3: Get a suggested fix from the LLM
        fix_response = get_llm_fix(default_nix_content, error_message)
        
        if fix_response:
            # Step 4: Apply the fix
            print("\n--- Applying LLM Fix ---")
            print("Explanation:\n", fix_response.explanation)
            write_file_content(initial_nix_path, fix_response.corrected_nix_code)
            
            # Step 5: Re-compile to verify the fix
            print("\n--- Re-compiling with new file ---")
            success_after_fix, new_error = nix_build_and_get_error(initial_nix_path)
            
            if success_after_fix:
                print("Compilation successful! The fix worked.")
            else:
                print("Compilation failed again with a new error (as expected in this mock scenario):")
                print(new_error)
                print("You would now loop this process with the new error message.")
    else:
        print("Initial build was successful or no error message could be retrieved.")
    
except FileNotFoundError as e:
    print(f"Fatal error: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


In [ ]:
# --- 4. Main Program Loop ---

def debug_loop(initial_nix_path: str, max_attempts: int = 3):
    """
    The main debugging loop.
    It repeatedly tries to build the Nix file, gets a fix from the LLM if it fails,
    and applies the fix, up to a maximum number of attempts.
    """
    attempts = 0
    build_succeeded = False
    error_message = ""
    
    while not build_succeeded and attempts < max_attempts:
        print(f"\n--- Attempt {attempts + 1}/{max_attempts} ---")
        
        try:
            default_nix_content = read_file_content(initial_nix_path)
            build_succeeded, error_message = nix_build_and_get_error(initial_nix_path)
            
            if build_succeeded:
                print("Final build successful! The Nix file is correct. 🎉")
                break
            
            print("Compilation Error:\n", error_message)
            
            # Get a suggested fix from the LLM
            print("\n--- Getting LLM Fix ---")
            fix_response = get_llm_fix(default_nix_content, error_message)
            
            if fix_response:
                # Apply the fix and overwrite the file
                print("\n--- Applying LLM Fix ---")
                print("Explanation:\n", fix_response.explanation)
                write_file_content(initial_nix_path, fix_response.corrected_nix_code)
            else:
                print("Could not get a fix from the LLM. Stopping.")
                break
                
        except FileNotFoundError as e:
            print(f"Fatal error: {e}. Please ensure the initial Nix file exists.")
            break
        except Exception as e:
            print(f"An unexpected error occurred: {e}")
            break
            
        attempts += 1
        
    if not build_succeeded:
        print("\n--- Final Status ---")
        print(f"Failed to fix the Nix file after {max_attempts} attempts.")
        print("Please review the 'default.nix' and 'error.log' for more details.")

# --- Call the main function with a mock setup ---
if __name__ == "__main__":
    # Create a directory for the output files
    output_dir = "output"
    os.makedirs(output_dir, exist_ok=True)
    
    # Start the debugging loop
    debug_loop(initial_nix_path)